In [2]:
import json
obj = [[1,2,3],123,123.123,'abc',{'key1':(1,2,3),'key2':(4,5,6)}] # 转换列表为JSON字符串
encodedjson = json.dumps(obj)
print(repr(obj))
print(encodedjson)
print(json.loads(encodedjson))

[[1, 2, 3], 123, 123.123, 'abc', {'key1': (1, 2, 3), 'key2': (4, 5, 6)}]
[[1, 2, 3], 123, 123.123, "abc", {"key1": [1, 2, 3], "key2": [4, 5, 6]}]
[[1, 2, 3], 123, 123.123, 'abc', {'key1': [1, 2, 3], 'key2': [4, 5, 6]}]


In [ ]:
json.dumps(12345678900) # 转换整数

In [ ]:
data = {'b':789,'c':456,'a':123}
print(json.dumps(data, sort_keys=True, indent=4))

In [ ]:
data = {'b':789,'c':456,'a':123}
json.dumps(data, separators=(',', ':'))

In [ ]:
json.dumps([1, 2, 3, {'4': 5, '6': 7}], separators=(',', ':')) 

In [ ]:
data = {'b':789,'c':456,(1,2):123}
print(json.dumps(data,skipkeys=True))

In [ ]:
def df(obj): 
    if isinstance(obj, complex): 
        return {'__complex__': True, 'real': obj.real, 'imag': obj.imag}
    return json.JSONEncoder.default(obj) 


data = json.dumps(1+2j, default=df)
print(data)

def as_complex(dct):
    try:
        if dct['__complex__']:
            return complex(dct['real'], dct['imag'])
    except KeyError:
        pass
    return dct

print(json.loads(data, object_hook=as_complex))

In [30]:
class Person(object):
    def __init__(self, name, age):
        self.name = name
        self.age = age
    def __repr__(self):
        return f"Person({self.name},{self.age})"
         
person = Person('ccc', 25)
    
def obj_to_dict(obj):
    dd ={}
    dd['__class__'] = obj.__class__.__name__
    dd['__module__'] = obj.__module__
    dd.update(obj.__dict__)
    return dd

print(obj_to_dict(person))


def dict_to_obj(dd):
    try:
        class_name = dd.pop('__class__')
        module_name = dd.pop('__module__')
        module = __import__(module_name, fromlist=[class_name])
        cls = getattr(module, class_name)
#         cls = globals()[class_name]
        instan = cls(**dd)
    except KeyError:
        instan = dd
    
    return instan
    
dict_to_obj(obj_to_dict(person))

dump = json.dumps(person, default=obj_to_dict)
print(dump)

load = json.loads(dump, object_hook=dict_to_obj)
print(load)

{'__class__': 'Person', '__module__': '__main__', 'name': 'ccc', 'age': 25}
{"__class__": "Person", "__module__": "__main__", "name": "ccc", "age": 25}
Person(ccc,25)


In [25]:
def obj_to_dict(obj):
    print(f'call obj_to_dict')
    dd ={}
    dd['__class__'] = obj.__class__.__name__
    dd['__module__'] = obj.__module__
    dd.update(obj.__dict__)
    return dd

class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        print(f"call MyEncoder's default")
        dd ={}
        dd['__class__'] = obj.__class__.__name__
        dd['__module__'] = obj.__module__
        dd.update(obj.__dict__)
        return dd

dump = json.dumps(person, cls=MyEncoder, default=obj_to_dict)
print(dump)


call obj_to_dict
{"__class__": "Person", "__module__": "__main__", "name": "ccc", "age": 25}


In [36]:
class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        dd ={}
        dd['__class__'] = obj.__class__.__name__
        dd['__module__'] = obj.__module__
        dd.update(obj.__dict__)
        return dd
    
class MyDecoder(json.JSONDecoder):
    def __init__(self):
        pass
        super().__init__(object_hook=self.dict_to_obj)
    
    def dict_to_obj(self,dd):
        try:
            class_name = dd.pop('__class__')
            module_name = dd.pop('__module__')
            module = __import__(module_name, fromlist=[class_name])
            cls = getattr(module, class_name)
            instan = cls(**dd)
        except KeyError:
            instan = dd

        return instan
    
dump = json.dumps(person, cls=MyEncoder)
print(dump)

# load = json.loads(dump, cls=MyDecoder, object_hook=dict_to_obj)  # 不能同时指定cls和object_hook，会出现冲突
load = json.loads(dump, cls=MyDecoder)
print(load)

{"__class__": "Person", "__module__": "__main__", "name": "ccc", "age": 25}
Person(ccc,25)
